In [6]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
import matplotlib.pyplot as matplot
%matplotlib inline 

In [7]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [8]:
# number of input, hidden and output nodes
input_nodes = 47628
hidden_nodes = 5000
output_nodes = 5

# learning rate
learning_rate = 0.35

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [9]:
#load the mnist training data CSV file into a list
training_data_file=open("train_100_20220522.csv",'r')
training_data_list=training_data_file.readlines()
training_data_list=training_data_list[1:]
training_data_file.close()

len(training_data_list)

100

In [10]:
#train the neural network
#go through all records on the training data set
for record in training_data_list:
     #split the record by the ',' commas
     all_values=record.split(',')
     #scale and shift the inouts
     inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01
     #create the target outout values (all 0.01, except the desired label which is 0.99)
     targets=numpy.zeros(output_nodes)+0.01
     #all_values[0] is the targetlabel for this record
     targets[int(all_values[0])]=0.99
     n.train(inputs,targets)
     pass

In [11]:
#epochs is the numbers of times the training data set is used for training

epochs=1
for e in range(epochs):
    #go through all records in the training data set
    for record in training_data_list:
        #split the record by the ',' commas
        all_values=record.split(',')
        #scale and shift the inputs
        inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01
        #create the target output values (all 0.01, except the desired label which is 0.99)
        targets=numpy.zeros(output_nodes)+0.01
        #all_values[0] is the taeget label for this record
        targets[int(all_values[0])]=0.99
        n.train(inputs,targets)
        pass
    pass

In [12]:
#load the mnist test data CSV file into a list
test_data_file=open("test_15_20220522.csv",'r')
test_data_list=test_data_file.readlines()
test_data_list=test_data_list[1:]
test_data_file.close()

In [13]:
#test the neural network

#scorecard for how well the network performs, initially empty
scorecard=[]
output_list = []

#go through all the records in the test data set
for record in test_data_list:
    #split the record by the ',' commas
    all_values=record.split(',')
    #correct answer is first value
    correct_label=int(all_values[0])
    print(correct_label,"correct label")
    #scale and shift the inputs
    inputs=(numpy.asfarray(all_values[1:])/255.0*0.99)+0.01
    #query the network
    outputs=n.query(inputs)
    
    output_list.append(outputs)
    
    label=numpy.argmax(outputs)
    print(label,"guessed answer")
    #append correct or incorrect to list
    if (label==correct_label):
        #network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        #network's answer does not match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    pass

2 correct label
3 guessed answer
2 correct label
3 guessed answer
4 correct label
3 guessed answer
0 correct label
3 guessed answer
3 correct label
3 guessed answer
4 correct label
3 guessed answer
0 correct label
3 guessed answer
3 correct label
3 guessed answer
2 correct label
3 guessed answer
4 correct label
3 guessed answer
3 correct label
3 guessed answer
1 correct label
3 guessed answer
1 correct label
3 guessed answer
1 correct label
3 guessed answer
0 correct label
3 guessed answer


In [14]:
#calculate the performance score, the fraction of correct answers
scorecard_array=numpy.asarray(scorecard)
print("performance=",scorecard_array.sum()/scorecard_array.size)

performance= 0.2


In [15]:
print(outputs)
print(output_list)

[[1.07781697e-21]
 [1.77730711e-26]
 [3.05077972e-25]
 [1.00000000e+00]
 [7.78076619e-24]]
[array([[1.07774589e-21],
       [1.77708427e-26],
       [3.05052964e-25],
       [1.00000000e+00],
       [7.77998101e-24]]), array([[1.07787274e-21],
       [1.77745166e-26],
       [3.05100023e-25],
       [1.00000000e+00],
       [7.78128728e-24]]), array([[1.07788303e-21],
       [1.77747318e-26],
       [3.05100565e-25],
       [1.00000000e+00],
       [7.78141205e-24]]), array([[1.07795409e-21],
       [1.77768187e-26],
       [3.05125731e-25],
       [1.00000000e+00],
       [7.78206654e-24]]), array([[1.07791613e-21],
       [1.77759268e-26],
       [3.05116223e-25],
       [1.00000000e+00],
       [7.78174199e-24]]), array([[1.07782452e-21],
       [1.77729588e-26],
       [3.05078316e-25],
       [1.00000000e+00],
       [7.78074580e-24]]), array([[1.07780943e-21],
       [1.77727740e-26],
       [3.05075487e-25],
       [1.00000000e+00],
       [7.78064217e-24]]), array([[1.07781809e